## NLP Setup

In [1]:
import random
import operator
import pickle
import sys
import spacy
from spacy import displacy
import benepar
import copy
import networkx as nx

In [2]:
# Set up and store NLP
nlp = spacy.load("en_core_web_md")
benepar.download('benepar_en3')
nlp.add_pipe('benepar', config={'model': 'benepar_en3'})

with open('./nlp/nlp.pickle', 'wb') as f:
    pickle.dump(nlp, f)

[nltk_data] Downloading package benepar_en3 to
[nltk_data]     C:\Users\regan\AppData\Roaming\nltk_data...
[nltk_data]   Package benepar_en3 is already up-to-date!


In [2]:
# Load NLP
#sys.path.append('../')

with open('./nlp/nlp.pickle', 'rb') as f:
    nlp = pickle.load(f)

In [3]:
from app.src.sentence_summarizer import SentenceSummarizer

summarizer = SentenceSummarizer(nlp)

In [4]:
from app.templates.meat_sale.deltas import meat_sale_deltas as deltas

s1 = deltas[0]
original_sentence = s1['original']
template_sentence = s1['general']
symboleo_template = s1['template'] 

print(f'- Original: {original_sentence}')
print(f'- General: {template_sentence}')
print(f'- Template: {symboleo_template}')

- Original: The Seller shall deliver the Order in one delivery within <delDueDateDays> days to the Buyer at its warehouse
- General: The Seller shall deliver the Order in one delivery [DELIVERY_TIMEFRAME] to the Buyer [DELIVERY_LOCATION]
- Template: delivery: O(seller, buyer, true, Happens(delivered))


In [5]:
from app.classes.spec.helpers import VariableDotExpression
from app.classes.spec.sym_event import VariableEvent, ContractEvent
from app.classes.spec.sym_point import PointFunction, PointExpression, PointAtomContractEvent, PointAtomParameterDotExpression
from app.classes.spec.predicate_function import PredicateFunctionHappens, PredicateFunctionWHappensBefore, PredicateFunctionHappensAfter
from app.classes.spec.helpers import TimeUnitStr, TimeValueInt


# The template should come from the deltas file. Store the obj, not the string
delivered = {
    'name': 'delivered'
}
delivered_vde = VariableDotExpression(delivered['name'])
delivered_event = VariableEvent(delivered_vde)
template = PredicateFunctionHappens(delivered_event)


In [6]:
from app.classes.spec.symboleo_spec import PAtom
from app.src.graph.graph_builder import GraphBuilder
from app.src.graph.graph_visualizer import GraphVisualizer
from app.src.dynamic_constructor import DynamicObjectConstructor
from app.src.rules.meat_sale.delivery_timeframe.matchers import get_matcher

graph_builder = GraphBuilder()
graph_visualizer = GraphVisualizer()

digraph = graph_builder.build(PAtom)

gv = graph_visualizer.create_viz(digraph.nodes)
gv.show('nx.html')

dynamic_constructor = DynamicObjectConstructor(digraph)

matcher = get_matcher(nlp)

In [15]:
#sentence = 'within 2 weeks'
sentence = 'after the third monday in april'
doc = nlp(sentence)
summarizer.summarize(sentence)


Sentence: after the third monday in april

i     TEXT            POS             TAG             DEP             LEMMA           HEAD            ENT            
--------------------------------------------------------------------------------------------------------------
0     after           ADP             IN              ROOT            after           after                          
1     the             DET             DT              det             the             monday                         
2     third           ADJ             JJ              amod            third           monday          ORDINAL        
3     monday          NOUN            NN              pobj            monday          after                          
4     in              ADP             IN              prep            in              monday                         
5     april           PROPN           NN              pobj            april           in              DATE           



            PP  

In [16]:
from app.src.rules.meat_sale.delivery_timeframe.delivery_timeframe_processor import DeliveryTimeframeProcessor

dtf = DeliveryTimeframeProcessor(template, matcher, nlp, dynamic_constructor)

results = dtf.process(doc)

print(f'Original: {template.to_sym()}')
print(f'New: {results[0].to_sym()}')

matches:
case matches: {}
primitices:
- <class 'app.classes.spec.sym_point.PointAtomContractEvent'> activated(self)
Original: Happens(delivered)


IndexError: list index out of range